Based on:

@book{leborgne2022fraud,

title={Reproducible Machine Learning for Credit Card Fraud Detection - Practical Handbook},

author={Le Borgne, Yann-A{\"e}l and Siblini, Wissam and Lebichot, Bertrand and Bontempi, Gianluca},

url={https://github.com/Fraud-Detection-Handbook/fraud-detection-handbook},

year={2022},

publisher={Universit{\'e} Libre de Bruxelles}

}

Covered subchapters:
* 7.2.3+ Feed-forward neural network

In [1]:
import torch
import datetime
import time
import numpy as np
import sklearn
from skorch import NeuralNetClassifier
import wandb


In [2]:
!curl -O https://raw.githubusercontent.com/Fraud-Detection-Handbook/fraud-detection-handbook/main/Chapter_References/shared_functions.py
%run shared_functions.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2 63257    2  1371    0     0   3902      0  0:00:16 --:--:--  0:00:16  3905
100 63257  100 63257    0     0   162k      0 --:--:-- --:--:-- --:--:--  162k


In [3]:
%run my_shared_functions.py

In [4]:
DIR_INPUT = '../fraud-detection-handbook/simulated-data-transformed/data/'

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: total: 578 ms
Wall time: 992 ms
919767 transactions loaded, containing 8195 fraudulent transactions


In [5]:
SEED = 42
seed_everything(SEED)

In [6]:
start_date_training = datetime.datetime.strptime("2018-07-25", "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7
delta_valid = delta_test

start_date_training_with_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))

(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)

(train_df, valid_df)=scaleData(train_df, valid_df, input_features)

In [7]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

Selected device is cuda


In [8]:
model = SimpleFraudMLP(len(input_features), 1000).to(DEVICE)

x_train = torch.FloatTensor(train_df[input_features].values)
x_valid = torch.FloatTensor(valid_df[input_features].values)
y_train = torch.FloatTensor(train_df[output_feature].values)
y_valid = torch.FloatTensor(valid_df[output_feature].values)

training_set = FraudDataset(x_train.to(DEVICE), y_train.to(DEVICE))
valid_set = FraudDataset(x_valid.to(DEVICE), y_valid.to(DEVICE))

training_generator,valid_generator = prepare_generators(training_set,valid_set,batch_size=64)

criterion = torch.nn.BCELoss().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.0005)

##### Early Stopping

In [9]:
config_mlp = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = 42,
    begin_date = '2018-07-25',
    delta_train = 7,
    delta_delay = 7,
    delta_test = 7,
    batch_size=64,
    num_workers=0,
    hidden_size = 1000,
    optimizer='sgd',
    lr=0.0005,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=500,
    scale=True,
    criterion='bce'
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config_mlp, tags=['mlp', 'imbalance-not-considered'])
config_mlp = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [10]:
model,training_execution_time,train_losses,valid_losses = training_loop_and_saving_best_wandb(model,training_generator,valid_generator,optimizer,criterion,
                                                            max_epochs=500,verbose=True, save_path='models/DL/mlp_earlystop/simple_mlp_model_earlystop.pt')


wandb: Network error (ConnectTimeout), entering retry loop.



Epoch 0: train loss: 0.18884140769858002
valid loss: 0.09599164529866104
New best score: 0.09599164529866104

Epoch 1: train loss: 0.09537467348799605
valid loss: 0.07040649621198455
New best score: 0.07040649621198455

Epoch 2: train loss: 0.07512162991938537
valid loss: 0.05619031202526503
New best score: 0.05619031202526503

Epoch 3: train loss: 0.06351821662434558
valid loss: 0.048948103641737434
New best score: 0.048948103641737434

Epoch 4: train loss: 0.058062408105261996
valid loss: 0.04489999113381563
New best score: 0.04489999113381563

Epoch 5: train loss: 0.05460313476310187
valid loss: 0.04229192821049304
New best score: 0.04229192821049304

Epoch 6: train loss: 0.05212243972213267
valid loss: 0.04033646309844389
New best score: 0.04033646309844389

Epoch 7: train loss: 0.05013998291259374
valid loss: 0.03883849721479625
New best score: 0.03883849721479625

Epoch 8: train loss: 0.04848510026577311
valid loss: 0.03766066508426952
New best score: 0.03766066508426952

Epoch 

In [11]:
wandb.log({'Training execution time': training_execution_time})
print(training_execution_time)
start_time=time.time()
# no need to set model in eval mode since there are no BN, Dropout layers
predictions_test = model(x_valid.to(DEVICE))
prediction_execution_time=time.time()-start_time
wandb.log({'Prediction execution time': prediction_execution_time})

7615.11332654953


In [12]:
predictions_df=valid_df
predictions_df['predictions']=predictions_test.detach().cpu().numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.866,0.613,0.624,0.284


In [13]:
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})

mlp_artifact = wandb.Artifact('mlp_earlystop', type='mlp', description='trained simple multilayer perceptron with 1 hidden layer and early stopping on')
mlp_artifact.add_dir('models/DL/mlp_earlystop')
wandb.log_artifact(mlp_artifact)
wandb.finish()

wandb: Adding directory to artifact (.\models\DL\mlp_earlystop)... Done. 0.0s


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
train loss,█▅▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val loss,█▅▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
AUC ROC,0.866
Average precision,0.613
Card Precision@100,0.284


##### ADAM optimizer

In [128]:
config_mlp = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = 42,
    begin_date = '2018-07-25',
    delta_train = 7,
    delta_delay = 7,
    delta_test = 7,
    batch_size=64,
    num_workers=0,
    hidden_size = 1000,
    optimizer='adam',
    lr=0.0005,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    criterion='bce'
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config_mlp, tags=['mlp', 'imbalance-not-considered'])
config_mlp = wandb.config

train loss,█▅▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val loss,█▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,0.02368
val loss,0.02089


In [129]:
seed_everything(SEED)
model = SimpleFraudMLP(len(input_features), 1000).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)
model,training_execution_time,train_losses_adam,valid_losses_adam = training_loop_and_saving_best_wandb(model,training_generator,valid_generator,optimizer,criterion,verbose=True,
                                                                        save_path='models/DL/mlp_adam/simple_mlp_model_adam.pt')


Epoch 0: train loss: 0.04304343708135103
valid loss: 0.022782339930685666
New best score: 0.022782339930685666

Epoch 1: train loss: 0.024800907236614312
valid loss: 0.020320492451187555
New best score: 0.020320492451187555

Epoch 2: train loss: 0.022864724756315968
valid loss: 0.019964506324946996
New best score: 0.019964506324946996

Epoch 3: train loss: 0.02190224805079865
valid loss: 0.020164376673846823
1  iterations since best score.

Epoch 4: train loss: 0.021138214083414146
valid loss: 0.019264124811276317
New best score: 0.019264124811276317

Epoch 5: train loss: 0.020224218546979124
valid loss: 0.01902117400453656
New best score: 0.01902117400453656

Epoch 6: train loss: 0.01973835375529052
valid loss: 0.019091747153019298
1  iterations since best score.

Epoch 7: train loss: 0.019350004048692814
valid loss: 0.018914142219476508
New best score: 0.018914142219476508

Epoch 8: train loss: 0.018840550201722144
valid loss: 0.01943170726620035
1  iterations since best score.

Epo

In [130]:
wandb.log({'Training execution time': training_execution_time})
print(training_execution_time)
start_time=time.time()
# no need to set model in eval mode since there are no BN, Dropout layers
predictions_test = model(x_valid.to(DEVICE))
prediction_execution_time=time.time()-start_time
wandb.log({'Prediction execution time': prediction_execution_time})
print(prediction_execution_time)

predictions_df=valid_df
predictions_df['predictions']=predictions_test.detach().cpu().numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

320.8815701007843
0.002000093460083008


,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.86,0.62,0.672,0.271


In [131]:
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})

mlp_artifact = wandb.Artifact('mlp_adam', type='mlp', description='trained simple multilayer perceptron with 1 hidden layer and adam optimizer')
mlp_artifact.add_dir('models/DL/mlp_adam')
wandb.log_artifact(mlp_artifact)
wandb.finish()

wandb: Adding directory to artifact (.\models\DL\mlp_adam)... Done. 0.0s


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
train loss,█▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val loss,█▄▃▄▂▂▂▂▂▂▂▄▂▂▁▂▂▃
AUC ROC,0.86
Average precision,0.62
Card Precision@100,0.271


##### Dropout

In [132]:
config_mlp = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = 42,
    begin_date = '2018-07-25',
    delta_train = 7,
    delta_delay = 7,
    delta_test = 7,
    batch_size=64,
    num_workers=0,
    hidden_size = 1000,
    optimizer='adam',
    lr=0.0005,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    dropout=0.2,
    criterion='bce'
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config_mlp, tags=['mlp', 'imbalance-not-considered'])
config_mlp = wandb.config

In [133]:
seed_everything(SEED)
model = SimpleFraudMLPWithDropout(len(input_features), 1000,0.2).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)
model,training_execution_time,train_losses,valid_losses = training_loop_and_saving_best_wandb(model,training_generator,valid_generator,optimizer,criterion,verbose=False,
                                                                                                        save_path='models/DL/mlp_dropout/simple_mlp_model_dropout.pt')

In [134]:
wandb.log({'Training execution time': training_execution_time})
print(training_execution_time)
model.eval()
start_time=time.time()
predictions_test = model(x_valid.to(DEVICE))
prediction_execution_time=time.time()-start_time
wandb.log({'Prediction execution time': prediction_execution_time})
print(prediction_execution_time)

predictions_df=valid_df
predictions_df['predictions']=predictions_test.detach().cpu().numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

196.85899806022644
0.0030002593994140625


,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.876,0.635,0.644,0.28


In [135]:
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})

mlp_artifact = wandb.Artifact('mlp_dropout', type='mlp', description='trained simple multilayer perceptron with 1 hidden layer and dropout')
mlp_artifact.add_dir('models/DL/mlp_dropout')
wandb.log_artifact(mlp_artifact)
wandb.finish()

wandb: Adding directory to artifact (.\models\DL\mlp_dropout)... Done. 0.0s


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
train loss,█▃▂▂▂▂▁▁▁▁▁
val loss,█▄▄▃▂▁▁▁▂▁▁
AUC ROC,0.876
Average precision,0.635
Card Precision@100,0.28


##### Embeddings

In [136]:
input_features

['TX_AMOUNT',
 'TX_DURING_WEEKEND',
 'TX_DURING_NIGHT',
 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
 'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW',
 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
 'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW',
 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
 'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW',
 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
 'TERMINAL_ID_RISK_1DAY_WINDOW',
 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
 'TERMINAL_ID_RISK_7DAY_WINDOW',
 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
 'TERMINAL_ID_RISK_30DAY_WINDOW']

In [137]:
def weekday(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    
    return int(weekday)

In [138]:
train_df['TX_WEEKDAY'] = train_df.TX_DATETIME.apply(weekday)
valid_df['TX_WEEKDAY'] = valid_df.TX_DATETIME.apply(weekday)
input_categorical_features = ['TX_WEEKDAY','TERMINAL_ID']

In [139]:
config_mlp = dict(
    dataset_id = 'fraud-detection-handbook-transformed-extended',
    validation = 'train test split',
    seed = 42,
    begin_date = '2018-07-25',
    delta_train = 7,
    delta_delay = 7,
    delta_test = 7,
    batch_size=64,
    num_workers=0,
    hidden_size = 1000,
    optimizer='adam',
    lr=0.0001,
    early_stopping=True,
    early_stopping_patience=2,
    max_epochs=100,
    scale=True,
    dropout=0.2,
    embedding_size=10,
    criterion='bce'
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config_mlp, tags=['mlp', 'imbalance-not-considered'])
config_mlp = wandb.config

In [140]:
seed_everything(SEED)
training_generator,valid_generator,categorical_inputs_modalities = prepare_generators_with_categorical_features(train_df,valid_df, input_features, input_categorical_features, output_feature, DEVICE, batch_size=64)

embedding_sizes = [10]*len(categorical_inputs_modalities)

model = FraudMLPWithEmbedding(categorical_inputs_modalities,len(input_features),embedding_sizes, 1000,0.2, DEVICE).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
model,training_execution_time,train_losses_embedding,valid_losses_embedding = training_loop_and_saving_best_wandb(model,training_generator,valid_generator,optimizer,criterion,verbose=True,
                                                                                            save_path='models/DL/mlp_embeddings/simple_mlp_model_embeddings.pt')


Epoch 0: train loss: 0.08502790267285948
valid loss: 0.029979193843052643
New best score: 0.029979193843052643

Epoch 1: train loss: 0.03248775874922047
valid loss: 0.024811104128242528
New best score: 0.024811104128242528

Epoch 2: train loss: 0.02725523496405545
valid loss: 0.02281399695258935
New best score: 0.02281399695258935

Epoch 3: train loss: 0.025421618136376157
valid loss: 0.021918508669915564
New best score: 0.021918508669915564

Epoch 4: train loss: 0.02447954625011795
valid loss: 0.02162936261819796
New best score: 0.02162936261819796

Epoch 5: train loss: 0.02352804730419906
valid loss: 0.021723798720363393
1  iterations since best score.

Epoch 6: train loss: 0.023122566236075826
valid loss: 0.021430913876725768
New best score: 0.021430913876725768

Epoch 7: train loss: 0.0225583291919388
valid loss: 0.021607249014746116
1  iterations since best score.

Epoch 8: train loss: 0.022182895909626212
valid loss: 0.021199198402019687
New best score: 0.021199198402019687

Epo

In [141]:
wandb.log({'Training execution time': training_execution_time})
print(training_execution_time)
model.eval()
x_valid_with_cat = prepare_x_valid_with_categorical_features(train_df, valid_df, input_features, input_categorical_features)
start_time=time.time()
predictions_test = model(x_valid_with_cat.to(DEVICE))
prediction_execution_time=time.time()-start_time
wandb.log({'Prediction execution time': prediction_execution_time})
print(prediction_execution_time)

predictions_df=valid_df
predictions_df['predictions']=predictions_test.detach().cpu().numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

183.27499628067017
0.004000186920166016


,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.838,0.602,0.605,0.28


In [142]:
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})

mlp_artifact = wandb.Artifact('mlp_embeddings', type='mlp', description='trained simple multilayer perceptron with 1 hidden layer and embedding layers')
mlp_artifact.add_dir('models/DL/mlp_embeddings')
wandb.log_artifact(mlp_artifact)
wandb.finish()

wandb: Adding directory to artifact (.\models\DL\mlp_embeddings)... Done. 0.0s


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
train loss,█▂▂▁▁▁▁▁▁▁▁▁
val loss,█▄▂▂▁▁▁▁▁▁▁▁
AUC ROC,0.838
Average precision,0.602
Card Precision@100,0.28


##### Prequential grid search

In [8]:
class FraudMLP(torch.nn.Module):
    
        def __init__(self, hidden_size=100,num_layers=1,p=0, input_size=len(input_features)):
            super(FraudMLP, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.p = p
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            
            self.fc_hidden=[]
            for _ in range(num_layers-1):
                self.fc_hidden.append(torch.nn.Linear(self.hidden_size, self.hidden_size))
                self.fc_hidden.append(torch.nn.ReLU())
                
            self.fc2 = torch.nn.Linear(self.hidden_size, 2)
            self.softmax = torch.nn.Softmax()
            
            self.dropout = torch.nn.Dropout(self.p)
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.relu(hidden)             
            hidden = self.dropout(hidden)
            
            for layer in self.fc_hidden:
                hidden=layer(hidden)
                hidden = self.dropout(hidden)
            
            output = self.fc2(hidden)
            output = self.softmax(output)
            
            return output

In [10]:
class FraudDatasetForPipe(torch.utils.data.Dataset):
    
    def __init__(self, x, y):
        'Initialization'
        self.x = torch.FloatTensor(x)
        self.y = None
        if y is not None:
            self.y = torch.LongTensor(y.values)
        

    def __len__(self):
        'Returns the total number of samples'
        return len(self.x)

    def __getitem__(self, index):
        'Generates one sample of data'
        if self.y is not None:
            # DON'T ADD .to(DEVICE) BELOW!!!
            # it will slow down training process more than 10 times
            # return self.x[index].to(DEVICE), self.y[index].to(DEVICE)
            return self.x[index], self.y[index]
        else:
            return self.x[index], -1       
            # return self.x[index].to(DEVICE), -1       

In [11]:
net = NeuralNetClassifier(
    FraudMLP,
    max_epochs=2,
    lr=0.001,
    optimizer=torch.optim.Adam,
    batch_size=64,
    dataset=FraudDatasetForPipe,
    iterator_train__shuffle=True,
    # device=DEVICE
)
net.set_params(train_split=False, verbose=0)

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.FraudMLP'>,
)

In [12]:
# Only keep columns that are needed as argument to custom scoring function
# to reduce serialization time of transaction dataset
transactions_df_scorer=transactions_df[['CUSTOMER_ID', 'TX_FRAUD','TX_TIME_DAYS']]

card_precision_top_100 = sklearn.metrics.make_scorer(card_precision_top_k_custom, 
                                                     needs_proba=True, 
                                                     top_k=100, 
                                                     transactions_df=transactions_df_scorer)

n_folds=4
start_date_training_for_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))
start_date_training_for_test = start_date_training+datetime.timedelta(days=(n_folds-1)*delta_test)
delta_assessment = delta_valid

Testing performance before the proper hp search

In [13]:
seed_everything(SEED)
start_time=time.time()

parameters = {
    'clf__lr': [0.001 ],
    'clf__batch_size': [64],
    'clf__max_epochs': [10, 20],
    'clf__module__hidden_size': [100],
    'clf__module__num_layers': [1,2],
    'clf__module__p': [0],
}

scoring = {'roc_auc':'roc_auc',
           'average_precision': 'average_precision',
           'card_precision@100': card_precision_top_100,
           }


performance_metrics_list_grid=['roc_auc', 'average_precision', 'card_precision@100']
performance_metrics_list=['AUC ROC', 'Average precision', 'Card Precision@100']

performances_df_validation=prequential_grid_search(
    transactions_df, net, 
    input_features, output_feature,
    parameters, scoring, 
    start_date_training=start_date_training_with_valid,
    n_folds=n_folds,
    expe_type='Validation',
    delta_train=delta_train, 
    delta_delay=delta_delay, 
    delta_assessment=delta_valid,
    performance_metrics_list_grid=performance_metrics_list_grid,
    performance_metrics_list=performance_metrics_list)

print("Validation: Total execution time: "+str(round(time.time()-start_time,2))+"s")

Validation: Total execution time: 82.48s


My execution time on GPU: ~70-80s

Handbook's execution time: 37.16s

Hp search:

In [13]:
seed_everything(SEED)


parameters = {
    'clf__lr': [0.001 , 0.0001, 0.0002],
    'clf__batch_size': [64,128,256],
    'clf__max_epochs': [10,20,40],
    'clf__module__hidden_size': [500],
    'clf__module__num_layers': [1,2],
    'clf__module__p': [0,0.2,0.4],
    'clf__module__input_size': [int(len(input_features))],
}

scoring = {'roc_auc':'roc_auc',
           'average_precision': 'average_precision',
           'card_precision@100': card_precision_top_100,
           }
           
performance_metrics_list_grid=['roc_auc', 'average_precision', 'card_precision@100']
performance_metrics_list=['AUC ROC', 'Average precision', 'Card Precision@100']

start_time=time.time()

performances_df=model_selection_wrapper(transactions_df, net, 
                                        input_features, output_feature,
                                        parameters, scoring, 
                                        start_date_training_for_valid,
                                        start_date_training_for_test,
                                        n_folds=n_folds,
                                        delta_train=delta_train, 
                                        delta_delay=delta_delay, 
                                        delta_assessment=delta_assessment,
                                        performance_metrics_list_grid=performance_metrics_list_grid,
                                        performance_metrics_list=performance_metrics_list,
                                        n_jobs=1)


execution_time_nn = time.time()-start_time

parameters_dict=dict(performances_df['Parameters'])
performances_df['Parameters summary']=[str(parameters_dict[i]['clf__lr'])+
                                   '/'+
                                   str(parameters_dict[i]['clf__batch_size'])+
                                   '/'+
                                   str(parameters_dict[i]['clf__max_epochs'])+
                                   '/'+
                                   str(parameters_dict[i]['clf__module__p'])+
                                   '/'+
                                   str(parameters_dict[i]['clf__module__num_layers'])
                                   for i in range(len(parameters_dict))]

performances_df_nn=performances_df

above search in the handbook took ~120min

my execution didn't finalize for over 600min...

nvidia-smi shows around 20% GPU usage of kernel

In [114]:
(train_df, test_df) = get_train_test_set(transactions_df,start_date_training,
                                       delta_train=7,delta_delay=7,delta_test=7)
(train_df, test_df)=scaleData(train_df, test_df, input_features)

seed_everything(SEED)

x_train = torch.FloatTensor(train_df[input_features].values)
x_test = torch.FloatTensor(test_df[input_features].values)
y_train = torch.FloatTensor(train_df[output_feature].values)
y_test = torch.FloatTensor(test_df[output_feature].values)

training_set = FraudDataset(x_train.to(DEVICE), y_train.to(DEVICE))
testing_set = FraudDataset(x_test.to(DEVICE), y_test.to(DEVICE))

training_generator,testing_generator = prepare_generators(training_set,testing_set,batch_size=64)

In [115]:
class FraudMLPHypertuned(torch.nn.Module):
    
        def __init__(self, input_size,hidden_size=500,num_layers=2,p=0.2):
            super(FraudMLPHypertuned, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.p = p
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            
            self.fc_hidden=[]
            for i in range(num_layers-1):
                self.fc_hidden.append(torch.nn.Linear(self.hidden_size, self.hidden_size).to(DEVICE))
                self.fc_hidden.append(torch.nn.ReLU())
                
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
            
            self.dropout = torch.nn.Dropout(self.p)
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.relu(hidden)             
            hidden = self.dropout(hidden)
            
            for layer in self.fc_hidden:
                hidden=layer(hidden)
                hidden = self.dropout(hidden)
            
            output = self.fc2(hidden)
            output = self.sigmoid(output)
            
            return output

In [119]:
config_mlp = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = 42,
    begin_date = '2018-07-25',
    delta_train = 7,
    delta_delay = 7,
    delta_test = 7,
    batch_size=64,
    num_workers=0,
    hidden_size = 500,
    num_hidden_layers = 2,
    optimizer='adam',
    lr=0.001,
    early_stopping=False,
    max_epochs=20,
    scale=True,
    dropout=0.2,
    criterion='bce'
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config_mlp, tags=['mlp', 'imbalance-not-considered', 'hypertuned'])
config_mlp = wandb.config

In [120]:
# Best hps taken from the handbook:
# learning_rate=0.001
# hidden layers 2
# hidden size 500
# batch_size 64
# max epochs 20
# dropout p 0.2

model = FraudMLPHypertuned(len(input_features)).to(DEVICE)
criterion = torch.nn.BCELoss().to(DEVICE)
model.eval()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

model,training_execution_time,train_losses,valid_losses = training_loop_and_saving_best_wandb(model,training_generator,testing_generator,optimizer,criterion,
                                                            apply_early_stopping=False, max_epochs=20,verbose=True, save_path='models/DL/mlp_hypertuned/mlp_hypertuned_model.pt')



Epoch 0: train loss: 0.05643073652563466
valid loss: 0.024018097617004054

Epoch 1: train loss: 0.02638936078540988
valid loss: 0.02153680467099957

Epoch 2: train loss: 0.024881045035415328
valid loss: 0.020263867595097056

Epoch 3: train loss: 0.02339501065342952
valid loss: 0.019642853217953907

Epoch 4: train loss: 0.022876052330221473
valid loss: 0.019326516251773885

Epoch 5: train loss: 0.022160829143681973
valid loss: 0.01904257496028656

Epoch 6: train loss: 0.02143366295834569
valid loss: 0.019655168659777535

Epoch 7: train loss: 0.021043319834753355
valid loss: 0.018881842694638867

Epoch 8: train loss: 0.0206349876358818
valid loss: 0.018905052699891872

Epoch 9: train loss: 0.020476300606205457
valid loss: 0.019224997351209826

Epoch 10: train loss: 0.019763412286676326
valid loss: 0.01863673846995786

Epoch 11: train loss: 0.02069019793411389
valid loss: 0.018626018677713935

Epoch 12: train loss: 0.019821298708868958
valid loss: 0.01858314455561718

Epoch 13: train los

In [121]:
wandb.log({'Training execution time': training_execution_time})
print(training_execution_time)
model.eval()
start_time=time.time()
predictions_test = model(x_test.to(DEVICE))
prediction_execution_time=time.time()-start_time
wandb.log({'Prediction execution time': prediction_execution_time})
print(prediction_execution_time)

predictions_df=test_df
predictions_df['predictions']=predictions_test.detach().cpu().numpy()
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

353.0805106163025
0.0034372806549072266


,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.876,0.659,0.686,0.284


In [122]:
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})

mlp_artifact = wandb.Artifact('mlp_hypertuned', type='mlp', description='trained hypertuned multilayer perceptron with 2 hidden layers')
mlp_artifact.add_dir('models/DL/mlp_hypertuned')
wandb.log_artifact(mlp_artifact)
wandb.finish()

wandb: Adding directory to artifact (.\models\DL\mlp_hypertuned)... Done. 0.0s


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
train loss,█▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val loss,█▅▄▃▃▂▃▂▂▂▂▂▂▂▁▁▂▁▁▁
AUC ROC,0.876
Average precision,0.659
Card Precision@100,0.284
